In [1]:
from tensorflow.keras.layers import Dense, Input, MaxPooling2D, Dropout, Flatten
from tensorflow.keras.models import Sequential
from tensorflow.keras.applications import resnet
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import tensorflow as tf
from tensorflow.keras import metrics
from tensorflow.keras.callbacks import EarlyStopping

In [2]:
batch_size = 64
width, height, channel = 400, 400, 3
no_epochs = 20
verbosity = 1
input_shape = (width, height, channel)

In [3]:
# ResNet50 모델 사용

pretrained_model = tf.keras.applications.ResNet50(
    include_top=False,
    weights="imagenet",
    input_tensor=None,
    input_shape=input_shape,
    pooling=None,
    classes=1000
)

In [4]:
train_datagen = ImageDataGenerator(rescale=1./255,
                                   shear_range=0.2,
                                   rotation_range=0.2,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True,
                                   vertical_flip=True,
                                   fill_mode='nearest')
train_generator = train_datagen.flow_from_directory('preprocessed_data/train',
                                                    target_size=(400, 400),
                                                    batch_size=batch_size,
                                                    class_mode='binary')

valid_datagen = ImageDataGenerator(rescale=1./255)
valid_generator = valid_datagen.flow_from_directory('preprocessed_data/test',
                                                    target_size=(400, 400),
                                                    batch_size=batch_size,
                                                    class_mode='binary')

pretrained_model.trainable=True
set_trainable=False

for layer in pretrained_model.layers:
    if layer.name == 'block7c_project_conv':
        set_trainable=True
    if set_trainable:
        layer.trainable=True
    else:
        layer.trainable=False


model=Sequential()
model.add(pretrained_model)
model.add(MaxPooling2D())
model.add(Dropout(0.2))
model.add(Flatten())
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=[metrics.AUC(), 'accuracy'])

Found 521 images belonging to 2 classes.
Found 135 images belonging to 2 classes.


In [5]:
es_callback = EarlyStopping(monitor='val_auc', mode='max', patience=8,
                                              verbose=1, min_delta=0.0001, restore_best_weights=True)

history = model.fit(train_generator,
                    steps_per_epoch=train_generator.samples//batch_size,
                    epochs = no_epochs,
                    validation_data=valid_generator,
                    validation_steps=valid_generator.samples//batch_size,
                    callbacks= [es_callback],
                    verbose=verbosity,
                   )

  ...
    to  
  ['...']
  ...
    to  
  ['...']
Train for 8 steps, validate for 2 steps
Epoch 1/20
8/8 [==============================] - 23s 3s/step - loss: 10.5304 - auc: 0.5347 - accuracy: 0.5208 - val_loss: 7.0162 - val_auc: 0.5000 - val_accuracy: 0.5000
Epoch 2/20
8/8 [==============================] - 18s 2s/step - loss: 5.6955 - auc: 0.6085 - accuracy: 0.5952 - val_loss: 5.9267 - val_auc: 0.5000 - val_accuracy: 0.5000
Epoch 3/20
8/8 [==============================] - 19s 2s/step - loss: 2.6627 - auc: 0.7243 - accuracy: 0.6696 - val_loss: 6.5825 - val_auc: 0.5000 - val_accuracy: 0.5000
Epoch 4/20
8/8 [==============================] - 19s 2s/step - loss: 1.7434 - auc: 0.7980 - accuracy: 0.7330 - val_loss: 5.1175 - val_auc: 0.5000 - val_accuracy: 0.5000
Epoch 5/20
8/8 [==============================] - 19s 2s/step - loss: 1.6949 - auc: 0.7894 - accuracy: 0.7090 - val_loss: 3.4716 - val_auc: 0.5000 - val_accuracy: 0.5000
Epoch 6/20
8/8 [==============================] - 19s 2s/st

In [6]:
prediction = model.predict(valid_generator)

In [7]:
import numpy as np

print(len(prediction[np.where(prediction > 0.5)]))
print(len(prediction[np.where(prediction < 0.5)]))

135
0


In [8]:
prediction

array([[0.99999905],
       [0.99999917],
       [0.99999917],
       [0.99999917],
       [0.99999905],
       [0.9999993 ],
       [0.99999917],
       [0.99999917],
       [0.99999917],
       [0.9999993 ],
       [0.99999917],
       [0.9999993 ],
       [0.99999917],
       [0.9999993 ],
       [0.99999905],
       [0.99999917],
       [0.9999993 ],
       [0.99999905],
       [0.99999917],
       [0.99999905],
       [0.9999993 ],
       [0.99999905],
       [0.9999993 ],
       [0.9999993 ],
       [0.9999993 ],
       [0.99999917],
       [0.99999905],
       [0.99999917],
       [0.99999917],
       [0.99999917],
       [0.9999993 ],
       [0.99999905],
       [0.9999993 ],
       [0.99999917],
       [0.99999917],
       [0.99999905],
       [0.99999917],
       [0.9999994 ],
       [0.9999993 ],
       [0.9999993 ],
       [0.9999993 ],
       [0.99999917],
       [0.99999917],
       [0.9999993 ],
       [0.99999905],
       [0.9999993 ],
       [0.99999917],
       [0.999